## Scratch Book to work out overview image

In [14]:
### COPY OF DEPENDENCIES AND SET UP FROM ADAPTABLE CODE BOOK

# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase

import os


import seaborn as sns
import sqlite3

### Set the folder paths for the data
### DATA FOLDER PATHS
data_folder = '../data/' # ROSTER DATA & GAME STATS
## IMAGES FOLDER PATHS
logo_folder = '../images/logos/' 
background_folder = '../images/backgrounds/'

# Roster Data Paths
roster_path_current = 'rosters/2023_master_roster.csv'
roster_path_previous = 'rosters/2022_master_roster.csv'
# Once the Database Clean and Transform code is updated to insert the roster, we can use the data from the database

# Game Stats Database Path
db_folder = '../data/'
# db_path = '2023_YTD_Game_Stats_Cleaned.db'
db_path = '2023_Season_10_31_23_Game_Stats.db'

## Connect to the Database
conn = sqlite3.connect(db_folder + db_path)

## Load rosters into dataframes
df_recent = pd.read_csv(data_folder + roster_path_current)
df_last = pd.read_csv(data_folder + roster_path_previous)


## HOTFIX#####
# Replace dashes with spaces in the School column of the roster dataframes
df_recent['School'] = df_recent['School'].str.replace('-', ' ')
df_last['School'] = df_last['School'].str.replace('-', ' ')


In [15]:
# TEST PARAMETERES

## Set paramaeters

### Set Matchup Tag - This will be used as a folder name to save the charts
matchup_tag = 'MSU_OSU_NEW' #

## Teams to Highlight - Use dashes instead of spaces
team_1 = 'Michigan State' # 
team_1_abrv = 'MSU' #
team_2 = 'Ohio State' #
team_2_abrv = 'OSU' #

## Path to School Logos
team_1_logo =  'msu.png' #
team_2_logo =  'osu.png' #
team_logos = {team_1: logo_folder + team_1_logo, team_2: logo_folder + team_2_logo}

## Set Colors
team_1_color =  'darkgreen' # For Michigan State
team_2_color =  'darkred' # For Ohio State
team_colors = {team_1: team_1_color, team_2: team_2_color}
############ Eventaully want to create a dictionary of colors for each team and save in a file to import

team_abrv_mapping = {
    'Michigan State': 'MSU',
    'Ohio State': 'OSU'
}


### Create a disctionary called highlight_colors to store the colors for each team
### Do it this way so i can reuse code from MSU_Explore_and_Plot notebook
highlight_colors = {team_1_abrv: team_1_color, team_2_abrv: team_2_color}

#### Set Background Image
path_to_bg_image =  'msu_osu_2.png' #
alpha_value = 0.1 # Set the transparency of the background image
## Load the background image
bg_img = mpimg.imread(background_folder + path_to_bg_image)



# Adjust the default chart size and aspect ratio
### SET THE FIGURE SIZE OF THE CHARTS ###
fig_deafult_1 = (10, 8) # Original
fig_deafult_2 = (12, 10)
# Turn on/off the figure size by commenting/uncommenting the lines below
figure_size = fig_deafult_1
# figure_size = fig_deafult_2


################################
## FONT PARAMETERES
font_title_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 36}
font_label_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 16}
font_tick_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 20}
font_label_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 20}
font_tick_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 16}
#################################





## Create a scatter plot of the age and in days of all

In [16]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl


## Compare The Rosters

In [17]:


## Function to compare the members of each team to the previous year and return information about roster turnover

from collections import Counter

def compare_rosters(df_recent, df_last, team):
    # Filter the dataframes by the team
    df_recent_team = df_recent[df_recent['School'] == team]
    df_last_team = df_last[df_last['School'] == team]
    
    # Set of player names for easier comparison
    recent_players = set(df_recent_team['Name'])
    last_players = set(df_last_team['Name'])
    
    # Calculate turnover stats
    new_players = recent_players - last_players
    departed_players = last_players - recent_players
    retained_players = recent_players.intersection(last_players)
    
    # Additional stats like turnover rate, etc.
    turnover_rate = len(departed_players) / len(last_players) * 100 if len(last_players) > 0 else 0
    
    # Count of players in each class rank for both data sets
    recent_class_count = Counter(df_recent_team['Yr.'])
    last_class_count = Counter(df_last_team['Yr.'])
    
    # Calculate how many players from each class were departed and retained
    departed_class_count = Counter(df_last_team[df_last_team['Name'].isin(departed_players)]['Yr.'])
    retained_class_count = Counter(df_last_team[df_last_team['Name'].isin(retained_players)]['Yr.'])
    
    return {
        'new_players': len(new_players),
        'departed_players': len(departed_players),
        'retained_players': len(retained_players),
        'turnover_rate': turnover_rate,
        'recent_class_count': recent_class_count,
        'last_class_count': last_class_count,
        'departed_class_count': departed_class_count,
        'retained_class_count': retained_class_count
    }


# Define a list of teams for analysis (you can replace this list with 'all_teams' or any specific list you have)
# teams_to_analyze = [team_1, team_2]

# Get unique team names from the roster data
all_teams = df_recent['School'].unique()
# Use the entire list of teams
teams_to_analyze = all_teams

# Initialize an empty list to store results
results_list = []

# Loop through each team and get the turnover stats
for team in teams_to_analyze:
    team_stats = compare_rosters(df_recent, df_last, team)
    team_stats['team'] = team  # Add team name to the result dictionary
    results_list.append(team_stats)

# Convert the list of dictionaries to a DataFrame
df_turnover_stats = pd.DataFrame(results_list)





df_turnover_stats.head()
# df_recent.head()

,new_players,departed_players,retained_players,turnover_rate,recent_class_count,last_class_count,departed_class_count,retained_class_count,team
0,13,11,23,32.352941,"{'Fr': 10, 'Gr': 5, 'Sr': 8, 'So': 11, 'Jr': 2}","{'Sr': 6, 'Jr': 9, 'So': 6, 'Fr': 11, 'Gr': 2}","{'So': 4, 'Gr': 2, 'Sr': 3, 'Jr': 2}","{'Sr': 3, 'Jr': 7, 'Fr': 11, 'So': 2}",American Intl
1,13,11,17,39.285714,"{'Fr': 12, 'Sr': 8, 'Jr': 8, 'So': 2}","{'Jr': 10, 'Sr': 8, 'So': 8, 'Fr': 2}","{'Jr': 2, 'Sr': 8, 'So': 1}","{'Jr': 8, 'So': 7, 'Fr': 2}",Lake Superior
2,11,9,20,31.034483,"{'Fr': 11, 'So': 9, 'Jr': 7, 'Sr': 4}","{'Fr': 10, 'Sr': 8, 'So': 7, 'Jr': 4}","{'Sr': 8, 'Fr': 1}","{'Fr': 9, 'So': 7, 'Jr': 4}",Army
3,12,13,15,46.428571,"{'Fr': 8, 'So': 6, 'Jr': 8, 'Sr': 3, 'Gr': 2}","{'Fr': 6, 'Sr': 9, 'Jr': 4, 'So': 8, 'Gr': 1}","{'Sr': 9, 'Jr': 2, 'Gr': 1, 'So': 1}","{'Fr': 6, 'So': 7, 'Jr': 2}",Bowling Green
4,14,12,16,42.857143,"{'Fr': 10, 'So': 8, 'Gr': 3, 'Jr': 6, 'Sr': 3}","{'Fr': 8, 'Gr': 5, 'Jr': 3, 'So': 10, 'Sr': 2}","{'Gr': 5, 'Jr': 1, 'Sr': 2, 'So': 4}","{'Fr': 8, 'So': 6, 'Jr': 2}",Vermont


In [18]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl


## Merge in data on average team age and experience

In [19]:
## Rename column on df_recent and _df_last to match the column on df_turnover_stats
df_recent = df_recent.rename(columns={'School': 'team'})
df_last = df_last.rename(columns={'School': 'team'})

# Calculate and store the avera age and experience level of each team
## Age - Use the DOB column to calculate each players age in data, average the age of each team, convert back to years (3 decimal places)
df_recent['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_recent['DOB'])).dt.days
df_recent['Age'] = df_recent['Age'] / 365


df_last['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_last['DOB'])).dt.days
df_last['Age'] = df_last['Age'] / 365


## Experience - Use the Yr. column to calculate each players experience in years, average the experience of each team, convert back to years (3 decimal places)
class_rank_mapping = {'Fr': 0, 'So': 1, 'Jr': 2, 'Sr': 3, 'Gr': 4}
df_recent['Experience'] = df_recent['Yr.'].map(class_rank_mapping)
df_last['Experience'] = df_last['Yr.'].map(class_rank_mapping)

# Caluclate the average age and experience of each team in each dataframe and store in a dictionary
team_age_dict_current = df_recent.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_current = df_recent.groupby('team')['Experience'].mean().round(3).to_dict()
team_age_dict_last = df_last.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_last = df_last.groupby('team')['Experience'].mean().round(3).to_dict()

## Add the average age and experience df_turnover_stats - Use the dictionaries created above, add current as age_current, add last as age_last
df_turnover_stats['age_current'] = df_turnover_stats['team'].map(team_age_dict_current)
df_turnover_stats['age_last'] = df_turnover_stats['team'].map(team_age_dict_last)

df_turnover_stats['exp_current'] = df_turnover_stats['team'].map(team_exp_dict_current)
df_turnover_stats['exp_last'] = df_turnover_stats['team'].map(team_exp_dict_last)


df_turnover_stats.head()

,new_players,departed_players,retained_players,turnover_rate,recent_class_count,last_class_count,departed_class_count,retained_class_count,team,age_current,age_last,exp_current,exp_last
0,13,11,23,32.352941,"{'Fr': 10, 'Gr': 5, 'Sr': 8, 'So': 11, 'Jr': 2}","{'Sr': 6, 'Jr': 9, 'So': 6, 'Fr': 11, 'Gr': 2}","{'So': 4, 'Gr': 2, 'Sr': 3, 'Jr': 2}","{'Sr': 3, 'Jr': 7, 'Fr': 11, 'So': 2}",American Intl,22.654,23.578,1.639,1.471
1,13,11,17,39.285714,"{'Fr': 12, 'Sr': 8, 'Jr': 8, 'So': 2}","{'Jr': 10, 'Sr': 8, 'So': 8, 'Fr': 2}","{'Jr': 2, 'Sr': 8, 'So': 1}","{'Jr': 8, 'So': 7, 'Fr': 2}",Lake Superior,22.354,23.971,1.400,1.857
2,11,9,20,31.034483,"{'Fr': 11, 'So': 9, 'Jr': 7, 'Sr': 4}","{'Fr': 10, 'Sr': 8, 'So': 7, 'Jr': 4}","{'Sr': 8, 'Fr': 1}","{'Fr': 9, 'So': 7, 'Jr': 4}",Army,22.462,23.563,1.129,1.345
3,12,13,15,46.428571,"{'Fr': 8, 'So': 6, 'Jr': 8, 'Sr': 3, 'Gr': 2}","{'Fr': 6, 'Sr': 9, 'Jr': 4, 'So': 8, 'Gr': 1}","{'Sr': 9, 'Jr': 2, 'Gr': 1, 'So': 1}","{'Fr': 6, 'So': 7, 'Jr': 2}",Bowling Green,22.083,23.479,1.444,1.679
4,14,12,16,42.857143,"{'Fr': 10, 'So': 8, 'Gr': 3, 'Jr': 6, 'Sr': 3}","{'Fr': 8, 'Gr': 5, 'Jr': 3, 'So': 10, 'Sr': 2}","{'Gr': 5, 'Jr': 1, 'Sr': 2, 'So': 4}","{'Fr': 8, 'So': 6, 'Jr': 2}",Vermont,21.913,23.045,1.367,1.500


## Add Draft Information to DF

In [20]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,team,Age,Experience
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl,20.369863,0
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl,25.057534,4
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl,23.295890,3
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl,20.580822,0
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl,22.241096,1


## Merge Draft Data Into DataFrame

In [21]:
# Create new column called "Drafted" and assign a value of 1 if the player was drafted, 0 if not
df_recent['Drafted'] = np.where(df_recent['Draft_Year'] > 0, 1, 0)
df_last['Drafted'] = np.where(df_last['Draft_Year'] > 0, 1, 0)



# Calculate the Draft Score Based on round drafted
### NEW DRAFT SCORE CALCULATION ###
# Assign weights to draft rounds
draft_weights = {
    1: 8,  # Extra weight for 1st round
    2: 6,
    3: 5,
    4: 4,
    5: 3,
    6: 2,
    7: 1
}

# Compute player draft scores and fill NaNs with 0
df_recent['Player_Draft_Score'] = df_recent['D_Round'].map(draft_weights).fillna(0)
df_last['Player_Draft_Score'] = df_last['D_Round'].map(draft_weights).fillna(0)

## Create new dataframes with team, count of drafted players, and sum of draft scores
df_recent_count = df_recent.groupby('team')['Drafted'].sum().reset_index()
df_recent_sum = df_recent.groupby('team')['Player_Draft_Score'].sum().reset_index()

df_last_count = df_last.groupby('team')['Drafted'].sum().reset_index()
df_last_sum = df_last.groupby('team')['Player_Draft_Score'].sum().reset_index()


# Merge the count and sum dataframes into single dataframe
df_recent_draft_score = df_recent_count.merge(df_recent_sum, on='team', how='left')
df_last_draft_score = df_last_count.merge(df_last_sum, on='team', how='left')

# Rename the columns
df_recent_draft_score = df_recent_draft_score.rename(columns={'Drafted': 'Drafted_Count', 'Player_Draft_Score': 'Player_Draft_Score_Sum'})
df_last_draft_score = df_last_draft_score.rename(columns={'Drafted': 'Drafted_Count_Previous', 'Player_Draft_Score': 'Player_Draft_Score_Sum_Previous'})

# Merge the two dataframes into one
df_draft_score = df_recent_draft_score.merge(df_last_draft_score, on='team', how='left')

# Normalize the two columns of draft scores to a 0-100 scale
df_draft_score['Normalized_Draft_Score'] = df_draft_score['Player_Draft_Score_Sum'] / df_draft_score['Player_Draft_Score_Sum'].max() * 100
df_draft_score['Normalized_Draft_Score_Previous'] = df_draft_score['Player_Draft_Score_Sum_Previous'] / df_draft_score['Player_Draft_Score_Sum_Previous'].max() * 100

# Calculate the change in draft score from last year
df_draft_score['Draft_Score_Change'] = df_draft_score['Normalized_Draft_Score'] - df_draft_score['Normalized_Draft_Score_Previous']
## Switch and store as percentage change
df_draft_score['Draft_Score_Change'] = df_draft_score['Draft_Score_Change'] / df_draft_score['Normalized_Draft_Score_Previous'] * 100

# Merge into the df_turnover_stats dataframe
df_turnover_stats = df_turnover_stats.merge(df_draft_score[['team', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']], on='team', how='left')

# Reorder columns so team is first
df_turnover_stats = df_turnover_stats[['team', 'new_players', 'departed_players', 'retained_players', 'turnover_rate', 'recent_class_count', 'last_class_count', 'departed_class_count', 'retained_class_count', 'age_current', 'age_last', 'exp_current', 'exp_last', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']]

# Calculate and store ranking for the Age and Experience columns
df_turnover_stats['age_rank_current'] = df_turnover_stats['age_current'].rank(ascending=False) # Oldest = 1 
df_turnover_stats['exp_rank_current'] = df_turnover_stats['exp_current'].rank(ascending=False) # Oldest = 1


## Save dataframe as csv in TEMP folder
df_turnover_stats.to_csv('../TEMP/df_turnover_stats.csv', index=False)

# df_turnover_stats.head(35)



## Calculate each teams current W/L/T Record from Database

In [22]:
# Query to get the required columns for calculating win/lose/tie records
query_linescore_for_records = """
SELECT Team, goalsT, Game_ID FROM linescore;
"""

# Execute the query and convert to a DataFrame
linescore_records_df = pd.read_sql(query_linescore_for_records, conn)

# Initialize empty dictionary to store win/lose/tie records
team_records = {}

# Loop through the DataFrame grouped by Game_ID to find winner, loser, or tie for each game
for game_id, game_data in linescore_records_df.groupby("Game_ID"):
    team1, team2 = game_data["Team"].values
    goals_team1, goals_team2 = game_data["goalsT"].values
    
    # Initialize the team record if not already present
    for team in [team1, team2]:
        if team not in team_records:
            team_records[team] = {"Wins": 0, "Losses": 0, "Ties": 0}
    
    # Determine the outcome of the game for each team
    if goals_team1 > goals_team2:
        team_records[team1]["Wins"] += 1
        team_records[team2]["Losses"] += 1
    elif goals_team2 > goals_team1:
        team_records[team2]["Wins"] += 1
        team_records[team1]["Losses"] += 1
    else:
        team_records[team1]["Ties"] += 1
        team_records[team2]["Ties"] += 1

# Convert the records dictionary to a DataFrame for better display
records_df = pd.DataFrame.from_dict(team_records, orient="index").reset_index()
records_df.rename(columns={"index": "Team"}, inplace=True)

records_df.tail()  # Display the first few rows
# len(records_df)  # Display the number of teams in the DataFrame

records_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    61 non-null     object
 1   Wins    61 non-null     int64 
 2   Losses  61 non-null     int64 
 3   Ties    61 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.0+ KB


In [23]:
print(df_turnover_stats.columns)

print(records_df.columns)

Index(['team', 'new_players', 'departed_players', 'retained_players',
       'turnover_rate', 'recent_class_count', 'last_class_count',
       'departed_class_count', 'retained_class_count', 'age_current',
       'age_last', 'exp_current', 'exp_last', 'Drafted_Count',
       'Drafted_Count_Previous', 'Draft_Score_Change',
       'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous',
       'age_rank_current', 'exp_rank_current'],
      dtype='object')
Index(['Team', 'Wins', 'Losses', 'Ties'], dtype='object')


In [24]:
## Merge into the df_turnover_stats dataframe
df_turnover_stats = df_turnover_stats.merge(records_df[['Team', 'Wins', 'Losses', 'Ties']], left_on='team', right_on='Team', how='left')

In [30]:
### Fuzzy Match the Team Names
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Create a list of all the uniqueteam names in the records_df
record_team_names = list(records_df['Team'].unique())

# Create a list of all the unique team names in the df_turnover_stats
turnover_team_names = list(df_turnover_stats['team'].unique())

# Sort both Lists alphabetically
record_team_names.sort()
turnover_team_names.sort()



## Print the length of each list and the first 5 items in each list
print(len(record_team_names))
print(len(turnover_team_names))
print(record_team_names[:5])
print(turnover_team_names[:5])

# # # Create an empty dataframe to store the fuzzy match results
# fuzzy_match_results = pd.DataFrame(columns=['turnover_team', 'records_Team', 'Score'])

# # Iterate through each team name in turnover_team_names
# for turnover_team in turnover_team_names:
#     # Use fuzzy matching to find the closest match in record_team_names
#     match, score = process.extractOne(turnover_team, record_team_names)
    
#     # Append the result to fuzzy_match_results DataFrame
#     fuzzy_match_results = fuzzy_match_results.append({
#         'turnover_team': turnover_team,
#         'records_Team': match,
#         'Score': score
#     }, ignore_index=True)

# # Display the fuzzy_match_results to see how well the team names matched
# print(fuzzy_match_results)



61
59
['AIC', 'ASU', 'Air Force', 'Alaska', 'Army']
['Air Force', 'Alaska', 'Alaska Anchorage', 'American Intl', 'Arizona State']


## Find The number of games that Do Not Have A Line Chart

In [ ]:
# Query to get the total number of unique games in the linescore table
query_total_games = "SELECT COUNT(DISTINCT Game_ID) FROM linescore;"
total_games = conn.execute(query_total_games).fetchone()[0] // 2  # Each game appears twice (one for each team)

# Query to get the total number of unique games in the line_chart table
query_total_games_line_chart = "SELECT COUNT(DISTINCT Game_ID) FROM line_chart;"
total_games_line_chart = conn.execute(query_total_games_line_chart).fetchone()[0]

# Calculate the percentage of games with incomplete line_chart data
percentage_incomplete = ((total_games - total_games_line_chart) / total_games) * 100

total_games, total_games_line_chart, percentage_incomplete



(68, 158, -132.35294117647058)

# Face Offs

In [ ]:
# Query to get the face-off related columns from the player_stats table
query_faceoff_data = """
SELECT Team, Player, FOW, FOL, Game_ID 
FROM player_stats
WHERE FOW IS NOT NULL AND FOL IS NOT NULL;
"""

# Execute the query and convert to a DataFrame
faceoff_data_df = pd.read_sql(query_faceoff_data, conn)

# Convert FOW and FOL to numeric types
faceoff_data_df["FOW"] = pd.to_numeric(faceoff_data_df["FOW"], errors='coerce')
faceoff_data_df["FOL"] = pd.to_numeric(faceoff_data_df["FOL"], errors='coerce')

# Calculate the overall face-off win/lose percentage for each team
team_faceoff_summary = faceoff_data_df.groupby("Team").agg({'FOW': 'sum', 'FOL': 'sum'}).reset_index()
team_faceoff_summary["Total_Attempts"] = team_faceoff_summary["FOW"] + team_faceoff_summary["FOL"]
team_faceoff_summary["Team_Win_Percentage"] = (team_faceoff_summary["FOW"] / team_faceoff_summary["Total_Attempts"]) * 100

team_faceoff_summary.head()  # Display the first few rows


,Team,FOW,FOL,Total_Attempts,Team_Win_Percentage
0,Air Force,278.0,221.0,499.0,55.711423
1,Alaska,165.0,168.0,333.0,49.549550
2,Alaska-Anchorage,235.0,245.0,480.0,48.958333
3,American Int'l,155.0,160.0,315.0,49.206349
4,Arizona State,172.0,173.0,345.0,49.855072


In [ ]:

# Execute the query and convert to a DataFrame
faceoff_data_df = pd.read_sql(query_faceoff_data, conn)

# Calculate the total face-off attempts for each player
faceoff_data_df["Total_Attempts"] = faceoff_data_df["FOW"] + faceoff_data_df["FOL"]
faceoff_data_df["Win_Percentage"] = (faceoff_data_df["FOW"] / faceoff_data_df["Total_Attempts"]) * 100

# Initialize an empty DataFrame to store the top 3 players for each team
top_players_50th_df = pd.DataFrame(columns=['Team', 'Player', 'Win_Percentage', 'Total_Attempts'])

# Loop through each team to calculate the 50th percentile threshold and find the top 3 players
for team, team_data in faceoff_data_df.groupby("Team"):
    threshold_50th = team_data["Total_Attempts"].quantile(0.5)
    filtered_players = team_data[team_data["Total_Attempts"] > threshold_50th]
    top_3_players = filtered_players.sort_values("Win_Percentage", ascending=False).head(3)[['Team', 'Player', 'Win_Percentage', 'Total_Attempts']]
    top_players_50th_df = pd.concat([top_players_50th_df, top_3_players])

top_players_50th_df.reset_index(drop=True, inplace=True)

# Reshape to wide format
top_players_wide_df = pd.DataFrame()

for team, team_data in top_players_50th_df.groupby("Team"):
    team_row = pd.DataFrame({"Team": [team]})
    for i, (_, row) in enumerate(team_data.iterrows()):
        team_row[f"Player_{i+1}"] = row["Player"]
        team_row[f"Player_{i+1}_Win_Percentage"] = row["Win_Percentage"]
        team_row[f"Player_{i+1}_Attempts"] = row["Total_Attempts"]
    top_players_wide_df = pd.concat([top_players_wide_df, team_row])

top_players_wide_df.reset_index(drop=True, inplace=True)

C:\Users\Justin\AppData\Local\Temp\ipykernel_5732\3328435781.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top_players_50th_df = pd.concat([top_players_50th_df, top_3_players])


In [ ]:
top_players_wide_df.head()  # Display the first few rows

,Team,Player_1,Player_1_Win_Percentage,Player_1_Attempts,Player_2,Player_2_Win_Percentage,Player_2_Attempts,Player_3,Player_3_Win_Percentage,Player_3_Attempts
0,Air Force,Clayton Cosentino,82.352941,17.0,Mason McCormick,78.571429,14.0,Clayton Cosentino,70.000000,20.0
1,Alaska,Harrison Israels,88.235294,17.0,Chase Dafoe,70.588235,17.0,Payton Matsui,69.230769,13.0
2,Alaska-Anchorage,Ben Almquist,82.352941,17.0,Adam Tisdale,78.571429,14.0,Riley Thompson,64.285714,14.0
3,American Int'l,Dustin Manz,66.666667,15.0,Julius Janhonen,62.500000,16.0,Julius Janhonen,60.869565,23.0
4,Arizona State,Brian Chambers,71.428571,21.0,Brian Chambers,65.217391,23.0,Ryan Alexander,63.636364,11.0


## merge into larger df

In [ ]:
#### Merge the faceoff dfs into the df_turnover_stats dataframe

df_faceoffs = team_faceoff_summary.merge(top_players_wide_df, on='Team', how='left')

# Add 'FO_' to the start of each column name except the 'Team' column make all column names lowercase

df_faceoffs.columns = ['Team'] + ['FO_' + col if col != 'Team' else col for col in df_faceoffs.columns[1:]]
df_faceoffs.columns = [col.lower() for col in df_faceoffs.columns]



df_faceoffs.head()

,team,fo_fow,fo_fol,fo_total_attempts,fo_team_win_percentage,fo_player_1,fo_player_1_win_percentage,fo_player_1_attempts,fo_player_2,fo_player_2_win_percentage,fo_player_2_attempts,fo_player_3,fo_player_3_win_percentage,fo_player_3_attempts
0,Air Force,278.0,221.0,499.0,55.711423,Clayton Cosentino,82.352941,17.0,Mason McCormick,78.571429,14.0,Clayton Cosentino,70.000000,20.0
1,Alaska,165.0,168.0,333.0,49.549550,Harrison Israels,88.235294,17.0,Chase Dafoe,70.588235,17.0,Payton Matsui,69.230769,13.0
2,Alaska-Anchorage,235.0,245.0,480.0,48.958333,Ben Almquist,82.352941,17.0,Adam Tisdale,78.571429,14.0,Riley Thompson,64.285714,14.0
3,American Int'l,155.0,160.0,315.0,49.206349,Dustin Manz,66.666667,15.0,Julius Janhonen,62.500000,16.0,Julius Janhonen,60.869565,23.0
4,Arizona State,172.0,173.0,345.0,49.855072,Brian Chambers,71.428571,21.0,Brian Chambers,65.217391,23.0,Ryan Alexander,63.636364,11.0


## Merge Faceoff data into larger dataframe

In [ ]:
# df_turnover_stats.head()

In [ ]:
## Merge the df_faceoffs dataframe into the df_turnover_stats dataframe

df_turnover_stats = df_turnover_stats.merge(df_faceoffs, on='team', how='left')


# Store the Large Dataframe under a new name
df_final = df_turnover_stats

# Save the dataframe as a csv file
df_final.to_csv('../TEMP/Team_Table with_Face_Off.csv', index=False)

print(df_final.shape)
print(df_final.columns)
# df_turnover_stats.head()
# df_turnover_stats.shape
# df_turnover_stats.describe()

(59, 37)
Index(['team', 'new_players', 'departed_players', 'retained_players',
       'turnover_rate', 'recent_class_count', 'last_class_count',
       'departed_class_count', 'retained_class_count', 'age_current',
       'age_last', 'exp_current', 'exp_last', 'Drafted_Count',
       'Drafted_Count_Previous', 'Draft_Score_Change',
       'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous',
       'age_rank_current', 'exp_rank_current', 'Team', 'Wins', 'Losses',
       'Ties', 'fo_fow', 'fo_fol', 'fo_total_attempts',
       'fo_team_win_percentage', 'fo_player_1', 'fo_player_1_win_percentage',
       'fo_player_1_attempts', 'fo_player_2', 'fo_player_2_win_percentage',
       'fo_player_2_attempts', 'fo_player_3', 'fo_player_3_win_percentage',
       'fo_player_3_attempts'],
      dtype='object')


# Rename Dataframe and Get to graphical portion

## Create a dataframe with very distint and styled  stats to output an easy to inteperate table or chart


In [ ]:
## Rename the above dataframe
summary_df = df_turnover_stats

# Calculate the ranks for 'age_current' and 'exp_current'  and Draft Score and add them as new columns to the DataFrame
# For Age and Experience, the lower the number, the higher the rank
# For Draft Score, the higher the number, the higher the rank
# Also Store the ranks as integers instead of floats

summary_df['age_rank_current'] = summary_df['age_current'].rank(ascending=False).astype(int)
summary_df['exp_rank_current'] = summary_df['exp_current'].rank(ascending=False).astype(int)
summary_df['draft_score_rank'] = summary_df['Normalized_Draft_Score'].rank(ascending=False).astype(int)
# make one for turnover rate as well - the higher the value the higher the ranking
summary_df['turnover_rate_rank'] = summary_df['turnover_rate'].rank(ascending=False).astype(int)

# Sort the DataFrame again based on 'team' for better visualization
summary_df = summary_df.sort_values(by='team')


keep_cols = ['team',  'Wins', 'Losses',   'Ties',
             'turnover_rate',  'turnover_rate_rank',
             'age_current', 'age_rank_current',
            'exp_current', 'exp_rank_current',
            'Drafted_Count', 'Normalized_Draft_Score']
summary_df = summary_df[keep_cols]


# Display the first few rows of the updated summary DataFrame
summary_df.head()



,team,Wins,Losses,Ties,turnover_rate,turnover_rate_rank,age_current,age_rank_current,exp_current,exp_rank_current,Drafted_Count,Normalized_Draft_Score
34,Air Force,2.0,4.0,0.0,34.482759,37,22.582,14,1.467,40,0,0.000000
6,Alaska,2.0,3.0,0.0,37.931034,25,22.819,5,1.483,39,0,0.000000
52,Alaska Anchorage,NaN,NaN,NaN,14.814815,57,22.643,12,1.387,44,0,0.000000
0,American Intl,NaN,NaN,NaN,32.352941,41,22.654,11,1.639,26,0,0.000000
44,Arizona State,NaN,NaN,NaN,48.275862,5,22.011,35,1.724,19,3,12.698413


In [ ]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 34 to 48
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   team                    59 non-null     object 
 1   Wins                    22 non-null     float64
 2   Losses                  22 non-null     float64
 3   Ties                    22 non-null     float64
 4   turnover_rate           59 non-null     float64
 5   turnover_rate_rank      59 non-null     int32  
 6   age_current             59 non-null     float64
 7   age_rank_current        59 non-null     int32  
 8   exp_current             59 non-null     float64
 9   exp_rank_current        59 non-null     int32  
 10  Drafted_Count           59 non-null     int32  
 11  Normalized_Draft_Score  59 non-null     float64
dtypes: float64(7), int32(4), object(1)
memory usage: 5.1+ KB
